<a href="https://colab.research.google.com/github/ematteli/VisionProject/blob/main/Classificazione_in_localizzabile_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usiamo ResNet50 per classificare le immagini in localizzabili oppure no

In [1]:
import os
import sys
from pathlib import Path

# with this line we can check if we are in colab or not
in_colab = 'google.colab' in sys.modules
print("are we in Colab?:",in_colab)

cwd = Path(os.getcwd())
if in_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    os.chdir('/gdrive/MyDrive/GeoEstimation')
    
else:
    #our defult wd in local should be Vision_Project
    if str(cwd)[-14:] == 'Vision_Project':
        os.chdir(cwd / 'GeoEstimation')
    sys.path.append(cwd / 'GeoEstimation')

are we in Colab?: True
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
✨🍰✨ Everything looks OK!


In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
import sys
from pathlib import Path
from PIL import Image
from numpy import asarray
import multiprocessing as mp

In [ ]:
dataset_test = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/im2gps3k_places365_classified.csv").dropna()

In [ ]:
# 4k dataset for the training
dataset = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/yfcc4k.csv").dropna()
indexNames = dataset[ dataset['LOC'] == 7 ].index
# Delete these row indexes from dataFrame
dataset.drop(indexNames , inplace=True)
wrong_row = dataset[dataset['IMG_ID'] == '2902900480(1).jpg'].index
dataset.drop(wrong_row, inplace=True)
set(dataset['LOC'])

{0.0, 1.0, 2.0}

In [ ]:
images_test = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]

In [ ]:
# 4k dataset for the training
images = ["/gdrive/MyDrive/GeoEstimation/resources/images/yfcc4k/{}".format(img) for img in dataset['IMG_ID']]

In [ ]:
#images = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]
#img_all = [Image.open(image) for image in images]
#img_data = [asarray(img) for img in img_all]

In [ ]:
# a function to load all the images in an array format in img_array
# other "easier solutions" were really slow, this works fine
import tensorflow as tf
from tqdm import tqdm
tf.config.set_visible_devices([], 'GPU')

def load_image(image_path):
    with Image.open(image_path) as img:
        img = img.resize((256, 256))
        return np.asarray(img)

# Crea una pool di processi
with mp.Pool() as pool:
  # Carica le immagini in parallelo
  images = tqdm(["/gdrive/MyDrive/GeoEstimation/resources/images/yfcc4k/{}".format(img) for img in dataset['IMG_ID']])
  img_data = pool.map(load_image, images)

# Crea un array multidimensionale a partire dalla lista di array numpy
img_array = np.stack(img_data)


100%|██████████| 4368/4368 [00:27<00:00, 161.01it/s]


In [ ]:
np.save(fout.npy, img_array)

In [ ]:
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/images4k_array.npy'
np.save(fout, img_array)

In [ ]:
# load
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/images4k_array.npy'
img_array = np.load(fout)

In [ ]:
# a function to load all the images in an array format in img_array
# other "easier solutions" were really slow, this works fine

# test set

import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

# Crea una pool di processi
with mp.Pool() as pool:
  # Carica le immagini in parallelo
  images_test = tqdm(["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset_test['IMG_ID']])
  img_data_test = pool.map(load_image, images_test)

# Crea un array multidimensionale a partire dalla lista di array numpy
img_array_test = np.stack(img_data_test)

100%|██████████| 2997/2997 [03:35<00:00, 13.88it/s]


In [ ]:
fout2 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/images_test3k_array.npy'
np.save(fout2, img_array_test)

In [ ]:
# load
fout2 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/images_test3k_array.npy'
img_array_test = np.load(fout2)

In [ ]:
# now we can re-set the GPU
import tensorflow as tf
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

In [ ]:
# the labels
label_array = np.array(dataset['LOC']).reshape((img_array.shape[0], 1))
# the labels
label_array_test = np.array(dataset_test['LOC']).reshape((img_array_test.shape[0], 1))

In [ ]:
labels_int = label_array.astype(int)

# Conto gli zeri, gli uni e i due nell'array
count = np.bincount(labels_int.flatten())

# Stampo i risultati
print(f"Zeri: {count[0]}")
print(f"Uni: {count[1]}")
print(f"Due: {count[2]}")

Zeri: 1821
Uni: 768
Due: 1779


In [ ]:
# train-test split
# X_train, X_test, y_train, y_test = train_test_split(img_array, label_array, test_size=0.16, random_state=123)

In [ ]:
X_train = img_array
y_train = label_array
X_test = img_array_test
y_test = label_array_test

In [ ]:
# processing data for one-hot encoding
def preprocess_data(X, Y):
  X_p = preprocess_input(X)
  Y_p = to_categorical(Y, 3)
  return X_p, Y_p

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
y_test = enc.fit_transform(y_test).toarray()
y_train = enc.fit_transform(y_train).toarray()

In [ ]:
# this
y_train = np.where(label_array != 0, 1, 0)
y_test = np.where(label_array_test != 0, 1, 0)

In [ ]:
# one hot encoding applied both to train and test sets
X_train, y_train = preprocess_data(X_train, y_train)
X_test, y_test = preprocess_data(X_test, y_test)

In [ ]:
# the model
input_t = Input(shape=(256, 256, 3))
res_model = ResNet50(include_top=False, #resnet50
                   weights='imagenet', #pretrained from imagenet
                   input_tensor=input_t)

In [ ]:
# set all the layers not-trainable
for layer in res_model.layers[:]:
  layer.trainable = False

In [ ]:
# add trainable layers to performe the classification
model = Sequential()
model.add(res_model)
model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.05),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 average_pooling2d (AverageP  (None, 7, 7, 2048)       0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dropout (Dropout)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1)                 100353    
                                                                 
Total params: 23,688,065
Trainable params: 100,353
Non-trainable params: 23,587,712
______________________________________

In [ ]:
pip install pyyaml h5py

In [ ]:
# checkpoints
checkpoint_path = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# checkpoints and fit
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(X_train, y_train, batch_size=32, epochs=8, callbacks=[cp_callback], verbose=1)
model.summary()

Epoch 1/8
137/137 [==============================] - ETA: 0s - loss: 66.3989 - accuracy: 0.7626
Epoch 1: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
137/137 [==============================] - 21s 129ms/step - loss: 66.3989 - accuracy: 0.7626
Epoch 2/8
137/137 [==============================] - ETA: 0s - loss: 29.7563 - accuracy: 0.8498
Epoch 2: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
137/137 [==============================] - 18s 128ms/step - loss: 29.7563 - accuracy: 0.8498
Epoch 3/8
137/137 [==============================] - ETA: 0s - loss: 15.4552 - accuracy: 0.9009
Epoch 3: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
137/137 [==============================] - 17s 125ms/step - loss: 15.4552 - accuracy: 0.9009
Epoch 4/8
137/137 [==============================] - ETA: 0s - loss: 13.9395 - accuracy: 0.9041
Epoch 4: saving model to /gdrive/MyDrive/GeoEstim

In [ ]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'cp.ckpt.index', 'checkpoint']

In [ ]:
# test
test_loss, test_acc = model.evaluate(X_test, y_test)

94/94 [==============================] - 12s 117ms/step - loss: 53.7093 - accuracy: 0.8208


In [ ]:
from sklearn.metrics import plot_roc_curve

In [ ]:
model.predict(X_test)

94/94 [==============================] - 12s 120ms/step


array([[1.],
       [1.],
       [0.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)

## 4k + 3k as train set

In [ ]:
X = np.concatenate((img_array, img_array_test))
y = np.concatenate((label_array, label_array_test))

In [ ]:
# this
y = np.where(y != 0, 1, 0)

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.16, 
                                                    random_state=123)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[cp_callback], verbose=1)
model.summary()

Epoch 1/10
194/194 [==============================] - ETA: 0s - loss: 60.8832 - accuracy: 0.7627
Epoch 1: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
194/194 [==============================] - 36s 125ms/step - loss: 60.8832 - accuracy: 0.7627
Epoch 2/10
193/194 [============================>.] - ETA: 0s - loss: 34.9793 - accuracy: 0.8347
Epoch 2: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
194/194 [==============================] - 24s 121ms/step - loss: 34.9900 - accuracy: 0.8343
Epoch 3/10
193/194 [============================>.] - ETA: 0s - loss: 25.9774 - accuracy: 0.8640
Epoch 3: saving model to /gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt
194/194 [==============================] - 24s 123ms/step - loss: 26.0041 - accuracy: 0.8640
Epoch 4/10
193/194 [============================>.] - ETA: 0s - loss: 21.4230 - accuracy: 0.8812
Epoch 4: saving model to /gdrive/MyDrive/GeoE

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

37/37 [==============================] - 6s 138ms/step - loss: 39.7302 - accuracy: 0.8338


### load model

In [ ]:
input_t = Input(shape=(256, 256, 3))
res_model = ResNet50(include_top=False, #resnet50
                   weights='imagenet', #pretrained from imagenet
                   input_tensor=input_t)
model = Sequential()
model.add(res_model)
model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.05),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 average_pooling2d_1 (Averag  (None, 7, 7, 2048)       0         
 ePooling2D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dropout_1 (Dropout)         (None, 100352)            0         
                                                                 
 dense_1 (Dense)             (None, 1)                 100353    
                                                                 
Total params: 23,688,065
Trainable params: 23,634,945
Non-trainable params: 53,120
_____________________________________

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

37/37 [==============================] - 7s 118ms/step - loss: 39.7302 - accuracy: 0.8338


In [ ]:
model.predict(X_test).round()

37/37 [==============================] - 4s 116ms/step


array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]], dtype=float32)

## Predict on the 22k

#### Chuncks

In [3]:
import tensorflow as tf
from tqdm import tqdm

In [7]:
dataset_25k = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/data25k_places365.csv").dropna()

In [4]:
def load_image(image_path):
    with Image.open(image_path) as img:
        img = img.resize((256, 256))
        return np.asarray(img)

In [5]:
def load_chunck(chunck):
  images_id = []
  with mp.Pool() as pool:
    # Carica le immagini in parallelo
    chunck_load = tqdm(["/gdrive/MyDrive/GeoEstimation/resources/images/new_data25k/{}".format(img) for img in chunck['IMG_ID']])
    chunck_data = pool.imap(load_image, chunck_load)
    #images_id.append(img)

  chunck_data = [img for img in chunck_data if len(img.shape) == 3 and img.shape[2] == 3]
  # Crea un array multidimensionale a partire dalla lista di array numpy
  chunck_array = np.stack(chunck_data)
  return chunck_array

In [ ]:
prova_ord = load_chunck(dataset_25k.iloc[0:1000,:])

  0%|          | 1/1000 [00:00<00:06, 165.66it/s]


In [ ]:
chunck1_array = load_chunck(dataset_25k.iloc[0:3000,:])

In [ ]:
chunck2_array = load_chunck(dataset_25k.iloc[3000:6000,:])

In [ ]:
chunck3_array = load_chunck(dataset_25k.iloc[6000:9000,:])

In [ ]:
chuncks_three_array = np.concatenate([chunck1_array, chunck2_array, chunck3_array], axis=0)
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/chuncks_three_array.npy'
np.save(fout, chuncks_three_array)

In [ ]:
chunck4_array = load_chunck(dataset_25k.iloc[9000:12000,:])

In [ ]:
chunck5_array = load_chunck(dataset_25k.iloc[12000:15000,:])

In [ ]:
chunck6_array = load_chunck(dataset_25k.iloc[15000:18000,:])

In [ ]:
chuncks_six_array = np.concatenate([chunck4_array, chunck5_array, chunck6_array], axis=0)
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/chuncks_six_array.npy'
np.save(fout, chuncks_six_array)

In [38]:
chunck7_array = load_chunck(dataset_25k.iloc[18000:20000,:])

100%|██████████| 2000/2000 [00:12<00:00, 156.19it/s]


In [39]:
chunck8_array = load_chunck(dataset_25k.iloc[20000:,:])

100%|██████████| 2354/2354 [00:12<00:00, 187.25it/s]


In [40]:
chuncks_eight_array = np.concatenate([chunck7_array, chunck8_array], axis=0)
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/chuncks_eight_array.npy'
np.save(fout, chuncks_eight_array)

In [ ]:
fout3 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/chuncks_three_array.npy'
fout6 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/chuncks_six_array.npy'
fout8 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/chuncks_eight_array.npy'
chunck_array_three = np.load(fout3)
chunck_array_six = np.load(fout6)
chunck_array_eight = np.load(fout8)

In [ ]:
img_array22k = np.concatenate([chunck_array_three, chunck_array_six, chunck_array_eight], axis=0)

In [ ]:
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/img_array22k.npy'
np.save(fout, img_array22k)

#### Load

In [ ]:
#dataset_25k = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/data25k_places365.csv").dropna()

In [2]:
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/img_array22k.npy'
img_array25k = np.load(fout)

In [64]:
#fout3 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/chuncks_three_array.npy'
#fout6 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/chuncks_six_array.npy'
fout8 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/chuncks_eight_array.npy'
#chunck_array_three = np.load(fout3)
#chunck_array_six = np.load(fout6)
chunck_array_eight = np.load(fout8)

In [51]:
chunck_array_three.shape

(8969, 256, 256, 3)

In [33]:
chunck_array_six.shape

(8972, 256, 256, 3)

In [42]:
chunck_array_eight.shape

(4334, 256, 256, 3)

In [43]:
8969+8972+4334

22275

In [65]:
import tensorflow as tf
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

In [66]:
input_t = Input(shape=(256, 256, 3))
res_model = ResNet50(include_top=False, #resnet50
                   weights='imagenet', #pretrained from imagenet
                   input_tensor=input_t)
model = Sequential()
model.add(res_model)
model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.05),
              metrics=['accuracy'])
model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 average_pooling2d (AverageP  (None, 7, 7, 2048)       0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dropout (Dropout)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1)                 100353    
                                                                 
Total params: 23,688,065
Trainable params: 23,634,945
Non

In [67]:
checkpoint_path = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/training_1/cp.ckpt'

In [68]:
model.load_weights(checkpoint_path)

In [70]:
y_pred25k = model.predict(chunck_array_eight)

136/136 [==============================] - 24s 107ms/step


In [71]:
fout = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/y_pred25k_check8.npy'
np.save(fout, y_pred25k)

In [72]:
fout3 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/y_pred25k_check3.npy'
fout6 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/y_pred25k_check6.npy'
fout8 = '/gdrive/MyDrive/GeoEstimation/checkpoints-ourResNet/y_pred25k_check8.npy'
y3 = np.load(fout3)
y6 = np.load(fout6)
y8 = np.load(fout8)

In [73]:
label22k = np.concatenate([y3, y6, y8], axis=0)

In [74]:
label22knew = (label22k>0.1).astype(int)

In [75]:
labels_int = label22knew

# Conto gli zeri, gli uni e i due nell'array
count = np.bincount(labels_int.flatten())

# Stampo i risultati
print(f"Zeri: {count[0]}")
print(f"Uni: {count[1]}")
#print(f"Due: {count[2]}")

Zeri: 8792
Uni: 13483


In [5]:
dataset_25k = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/data25k_places365.csv").dropna()

In [52]:
import os
import pandas as pd
from PIL import Image
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

def is_color(filename):
    file_path = os.path.join(image_dir, filename)
    with Image.open(file_path) as img:
        img = np.asarray(img)
        return (len(img.shape) == 3 and img.shape[2] == 3)

def process_chunk(chunk):
    chunk_images = []
    for filename in chunk:
        if filename.endswith(".jpg"):
            if is_color(filename):
                image_id = os.path.splitext(filename)[0]
                chunk_images.append(image_id)
    return chunk_images

data_file = "/gdrive/MyDrive/GeoEstimation/resources/data25k_places365.csv"
image_dir = "/gdrive/MyDrive/GeoEstimation/resources/images/new_data25k"

data = pd.read_csv(data_file).dropna()

image_filenames = os.listdir(image_dir)
num_cpus = cpu_count()
chunk_size = len(image_filenames) // num_cpus
chunks = [image_filenames[i:i+chunk_size] for i in range(0, len(image_filenames), chunk_size)]

with Pool(processes=num_cpus) as pool:
    results = []
    for chunk_result in tqdm(pool.imap(process_chunk, chunks), total=len(chunks)):
        results.append(chunk_result)

color_images = [id for chunk in results for id in chunk]
data = data[data['IMG_ID'].isin(color_images)]


100%|██████████| 3/3 [02:56<00:00, 58.77s/it] 


In [56]:
color_imagesJPG = [color_image + '.jpg' for color_image in color_images]

In [61]:
#data = pd.read_csv(data_file).dropna()

In [62]:
data = data[data['IMG_ID'].isin(color_imagesJPG)]

In [78]:
data['pred_LOC'] = label22knew

In [79]:
data

,index,IMG_ID,AUTHOR,LAT,LON,S3_Label,S16_Label,S365_Label,Prob_indoor,Prob_natural,Prob_urban,url,pred_LOC
0,0,45-77-3228511372.jpg,43008843@N00,46.778991,23.611207,0,5,21,9.786990e-01,0.001071,0.020230,https://live.staticflickr.com/3451/3228511372_...,0
1,1,2e-b5-4325164013.jpg,22489773@N02,40.445379,116.663990,2,14,201,1.687362e-02,0.366694,0.616432,https://live.staticflickr.com/4021/4325164013_...,1
2,2,68-42-2195251385.jpg,85976263@N00,47.469907,19.072952,0,5,250,9.645445e-01,0.000001,0.035454,https://live.staticflickr.com/2246/2195251385_...,0
3,3,87-7b-6786398493.jpg,70806177@N00,31.704666,35.206666,0,5,169,9.125900e-01,0.005846,0.081565,https://live.staticflickr.com/7159/6786398493_...,1
4,4,8b-a2-5663163551.jpg,46604778@N00,32.654696,-16.907958,2,14,268,1.941495e-01,0.118745,0.687105,https://live.staticflickr.com/5268/5663163551_...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22349,25595,17-76-4774603929.jpg,80547277@N00,45.484977,-123.972977,2,7,116,3.012353e-02,0.461205,0.508672,https://live.staticflickr.com/4096/4774603929_...,0
22350,25596,8c-76-6576441431.jpg,17373191@N00,41.765198,-74.161920,2,9,340,1.525349e-02,0.482444,0.502303,https://live.staticflickr.com/7012/6576441431_...,1
22351,25597,db-a5-2126931545.jpg,9456163@N05,29.422394,-98.488590,1,6,277,2.456120e-02,0.783856,0.191583,https://live.staticflickr.com/2060/2126931545_...,1
22352,25598,14-bf-2419255338.jpg,54177777@N00,37.563710,-122.513260,1,6,48,7.339240e-07,0.999492,0.000508,https://live.staticflickr.com/2415/2419255338_...,0


In [80]:
data.to_csv("/gdrive/MyDrive/GeoEstimation/resources/reduced_places365.csv")